### Make sure to use Python3, since it has natural order in dictionaries

In [2]:
import sys

print(sys.executable)

/data/anaconda3/bin/python


### Control client

In [3]:
import requests

from requests_kerberos import HTTPKerberosAuth, DISABLED
from requests.status_codes import codes


class ControlClient:
    _RESULTS = "results"
    _COUNT = "count"
    _PAGE_SIZE = "page_size"

    def __init__(self, base_url):
        self.base_url = base_url
        self._auth = HTTPKerberosAuth(mutual_authentication=DISABLED)

    def get(self, url, **kwargs):
        return self._request('get', url, **kwargs)

    def get_object_by_name(self, url, name, to_field="name", **kwargs):
        if "params" not in kwargs:
            kwargs["params"] = {}
        params = {to_field: name, self._PAGE_SIZE: 1}
        kwargs["params"].update(**params)

        response = self.get(url, **kwargs)

        if response.status_code != codes.ALL_OK:
            raise ValueError("Invalid request: {}".format(response.text))

        try:
            parsed = response.json()
        except Exception as e:
            raise ValueError(
                "Got invalid json from response: {}. "
                "Original response test: {}".format(e, response.text)
            )

        if parsed[self._COUNT] == 0:
            raise ValueError("Object with name '{}' does not exist.".format(name))

        if parsed[self._COUNT] > 1:
            raise ValueError("Got multiple objects with name '{}'.".format(name))

        return parsed[self._RESULTS][0]

    def post(self, url, **kwargs):
        return self._request('post', url, **kwargs)

    def patch(self, url, **kwargs):
        return self._request('patch', url, **kwargs)

    def delete(self, url, **kwargs):
        return self._request('delete', url, **kwargs)

    def _request(self, method, url, **kwargs):
        # cast absolute to relative
        if url.startswith("/"):
            url = url[1:]

        full_url = requests.compat.urljoin(self.base_url, url)
        kwargs['auth'] = self._auth
        try:
            return requests.request(method, full_url, **kwargs)
        except Exception as e:
            raise ValueError(
                "Got error making HTTP {} request "
                "on url '{}', original error message: {}.".format(method.upper(), full_url, e)
            )


cc = ControlClient("http://dmcontrol.host/api/")
cc_dev = ControlClient("http://dmcontrol-dev.host/api/")

### Basic tools for migrations

In [4]:
import os
import dill
import tqdm
import datetime
import subprocess


class MigrationUtils(object):

    def __init__(self, prod=False, migration_id=None):
        migration_id = migration_id or datetime.datetime.now().strftime("%Y%m%d%H%M%S%f")
        self._xapp_cache = {}

        if prod:
            self.client = ControlClient("http://dmcontrol.host/api/")
            self._ctid = "ctid__prod_{}"
            self._filepath = "migration_prod_{}".format(migration_id)
        else:
            self.client = ControlClient("http://dmcontrol-dev.host/api/")
            self._ctid = "ctid__dev_{}"
            self._filepath = "migration_dev_{}".format(migration_id)

    @staticmethod
    def check_response(r, status):
        if r.status_code != status:
            raise Exception(
                "Bad request for url={} - status={}, reason='{}'".format(r.url, r.status_code, r.reason)
            )

    def get_user_projects(self):
        response = self.client.get(url="compose/project/", params={"page_size": 1000000})
        self.check_response(response, 200)
        projects = response.json()["results"]

        response = self.client.get(url="target/portrait/", params={"page_size": 1000000})
        self.check_response(response, 200)
        portraits = response.json()["results"]

        response = self.client.get(url="target/lookalike/", params={"page_size": 1000000})
        self.check_response(response, 200)
        lookalikes = response.json()["results"]

        response = self.client.get(url="target/sales_report/", params={"page_size": 1000000})
        self.check_response(response, 200)
        sales_reports = response.json()["results"]

        managed_project_names = set.union(
            set(_["project"] for _ in portraits),
            set(_["project"] for _ in lookalikes),
            set(_["project"] for _ in sales_reports)
        )

        return [p for p in projects if p["name"] not in managed_project_names]

    def disable_projects(self, projects):
        if os.path.exists(self._filepath):
            raise AssertionError("Migration file '{}' already exists".format(self._filepath))

        enabled_project_ids = []

        for p in tqdm.tqdm_notebook(projects, total=len(projects)):
            if p["enabled"]:
                enabled_project_ids.append(p["id"])
                response = self.client.patch(url="compose/project/{}/".format(p["id"]), json={"enabled": False})
                self.check_response(response, 200)

        print("There was {} enabled projects".format(len(enabled_project_ids)))

        with open(self._filepath, "wb") as fd:
            dill.dump(enabled_project_ids, fd)

    def enable_projects(self, project_ids=None):
        if project_ids is None:
            with open(self._filepath, "rb") as fd:
                project_ids = dill.load(fd)

        for p_id in tqdm.tqdm_notebook(project_ids, total=len(project_ids)):
            response = self.client.patch(url="compose/project/{}/".format(p_id), json={"enabled": True})
            self.check_response(response, 200)

    def update_pipelines(self, project_name, kill_in_yarn=True):
        if kill_in_yarn:
            response = self.client.get(
                url="compose/jobrun/",
                params={"project": project_name, "status": "running", "page_size": 1000000}
            )
            self.check_response(response, 200)

            for jobrun in response.json()["results"]:
                self.kill_yarn_apps(ctid=self._ctid.format(jobrun["task"]))

        response = self.client.get(
            url="compose/pipeline/",
            params={"project": project_name, "page_size": 1000000}
        )
        self.check_response(response, 200)

        for pipeline in response.json()["results"]:
            if pipeline["status"] in {"pending", "running"}:
                response = self.client.delete(url="compose/pipeline/{}/".format(pipeline["id"]))
                self.check_response(response, 204)

                response = self.client.post(
                    url="compose/pipeline/",
                    json={
                        "project": project_name,
                        "tag": pipeline["tag"],
                        "patch": pipeline["patch"],
                        "force": pipeline["force"],
                        "keep": pipeline["keep"]
                    }
                )
                self.check_response(response, 201)

    @staticmethod
    def kill_yarn_apps(ctid):
        all_apps = subprocess.Popen(["yarn", "application", "-list"], stdout=subprocess.PIPE)
        p = subprocess.Popen(
            ["grep", ctid], stdin=all_apps.stdout, stdout=subprocess.PIPE, stderr=subprocess.PIPE
        )
        _stdout, _stderr = p.communicate()
        _stdout = _stdout.strip().decode("utf-8")
        if _stdout:
            for row in _stdout.split("\n"):
                app_id = row.strip().split("\t")[0].strip()
                cmd = ["yarn", "application", "-kill", app_id]
                p = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                _stdout, _stderr = p.communicate()
                print(
                    "COMMAND: {}\n\tSTDOUT: {}\tSTDERR: {}\tRETURN CODE: {}".format(
                        " ".join(cmd), _stdout, _stderr, p.returncode
                    )
                )

    def prettify_job_config(self, job):
        """Just reorder top-level job parameters."""
        xapp = self._xapp_cache.get(job["xapp"])

        if xapp is None:
            response = self.client.get(url="compose/xapp/", params={"name": job["xapp"]})
            self.check_response(response, 200)
            xapp = response.json()["results"][0]
            self._xapp_cache[job["xapp"]] = xapp

        job_config = {}

        for param_name, param_type in xapp["config"]["schema"]["properties"].items():
            if param_name in job["config"]:
                job_config[param_name] = job["config"][param_name]
            elif "default" not in param_type:  # Doesn't work with $defs yet
                raise ValueError(
                    "There is no mandatory parameter '{}' in job for Xapp='{}'".format(param_name, job["xapp"])
                )

        return job_config

## Migration steps

In [5]:
# Create migration instance
# migration = MigrationUtils(prod=True)

In [6]:
# Get all user-defined projects
user_projects = migration.get_user_projects()
print(len(user_projects))

621


In [12]:
# TODO: add dry-run mode
# TODO: add rollback option: snapshot(), restore(snapshot)
# TODO: tqdm dependency should be soft

### I. Switch off
Manual actions:
0. Remember (write down ids) originally enabled managers (https://dmcontrol-dev.host/admin/target/targetmanagermodel/).
1. For all managers - set `project.enabled: false` in manager config (e.g https://dmcontrol-dev.host/admin/target/targetmanagermodel/5/change/#/tab/module_1/).
2. Sync (sync connected ... GO).
3. Switch off (disable) managers.
4. Start code below to switch off all user projects.

There must be also a flag from https://jira.host/browse/TRG-72710 (https://dmcontrol.host/admin/compose/settingmodel/ `ENGINEERING_WORK: true`)

In [8]:
migration.disable_projects(user_projects)


There was 529 enabled projects


### II. Migration
Manual actions:
1. Merge prj git branch (to dev or to master).
2. Update Xapp schemas in Control.
3. Start migration for user projects (run code below).
4. Update target manager configs in Control; Sync ... GO.
5. Start update pipelines for managed projects related to TargetObjects at non-terminal status (code below).  

In [9]:
# Get new config for a single project.
# Here is an essence of migration.

import os
import copy

# TODO: add checks, new values should be not empty, etc.
def get_new_config(project_config):
    """It is specifically for current migration changes."""

    project_config = copy.deepcopy(project_config)

    for job_name, job in project_config["jobs"].items():

        if job["xapp"] == "Apply":
            job["config"].pop("source_dt", None)

        elif job["xapp"] == "CombineAdFeatures":
            max_dt_diff = job["config"].pop("max_dt_diff", None)

            if max_dt_diff is not None:
                job["config"]["period"] = max_dt_diff + 1
                job["config"]["dt_selection_mode"] = "single_last"
            # Else we degrade to new defaults, i.e. ~ max_dt_diff=0, just because we can

        elif job["xapp"] == "CombineUniversalFeatures":
            job["config"]["period"] = job["config"].pop("max_dt_diff") + 1
            job["config"]["dt_selection_mode"] = "single_last"

        elif job["xapp"] == "ExportAdFeatures":
            target_hdfs_basedir = job["config"].pop("target_hdfs_basedir")
            job["config"]["features_subdir"] = os.path.basename(os.path.normpath(target_hdfs_basedir))
            
            features = job["config"]["export_columns"].pop("features")
            job["config"]["export_columns"]["features"] = [
                {"name": name, "expr": expr} for name, expr in features.items()
            ]

        elif job["xapp"] == "ExportAudienceRb":
            job["config"].pop("target_hdfs_basedir")

        elif job["xapp"] == "ExportAudienceTrg":
            job["config"].pop("target_hdfs_basedir")

            # Current projects only win because of this
            job["config"].pop("min_score", None)
            job["config"].pop("max_score", None)

        elif job["xapp"] == "ExportUniversalFeatures":
            job["config"].pop("target_hdfs_basedir")

            feature_name = job["config"].pop("feature_name")
            job["config"]["features_subdir"] = feature_name
            job["config"]["export_columns"]["features"] = [
                {"name": feature_name, "expr": job["config"]["export_columns"].pop("feature")}
            ]

        elif job["xapp"] == "CustomFinder":
            for _, segment_config in job["config"]["audience"]["segments"].items():
                # It is better to set it explicitly
                segment_config["period"] = segment_config.get("period", 1)

                if segment_config.pop("only_last_dt", False):
                    segment_config["dt_selection_mode"] = "single_last"
                else:
                    segment_config["dt_selection_mode"] = "multiple_any"

        # Final config prettification
        job["config"] = migration.prettify_job_config(job)

    return project_config

In [10]:
# Migration for user projects
for p in tqdm.tqdm_notebook(user_projects, total=len(user_projects)):
    try:
        new_config = get_new_config(p["config"])
        if new_config != p["config"]:
            response = migration.client.patch(
                url="compose/project/{}/".format(p["id"]),
                json={"config": new_config}
            )
            migration.check_response(response, 200)

            migration.update_pipelines(project_name=p["name"], kill_in_yarn=True)

    except Exception as e:
        print("There is a trouble with project='{}': {}".format(p["name"], e))

COMMAND: yarn application -kill application_1638133298630_25555
	STDOUT: b'Killing application application_1638133298630_25555\n'	STDERR: b'21/12/01 16:06:38 INFO client.AHSProxy: Connecting to Application History server at rbhp-control5.rbdev.mail.ru/10.123.0.19:10200\n21/12/01 16:06:38 INFO impl.YarnClientImpl: Killed application application_1638133298630_25555\n'	RETURN CODE: 0
COMMAND: yarn application -kill application_1638133298630_25972
	STDOUT: b'Killing application application_1638133298630_25972\n'	STDERR: b'21/12/01 16:06:42 INFO client.AHSProxy: Connecting to Application History server at rbhp-control5.rbdev.mail.ru/10.123.0.19:10200\n21/12/01 16:06:43 INFO impl.YarnClientImpl: Killed application application_1638133298630_25972\n'	RETURN CODE: 0
COMMAND: yarn application -kill application_1638133298630_25695
	STDOUT: b'Killing application application_1638133298630_25695\n'	STDERR: b'21/12/01 16:07:35 INFO client.AHSProxy: Connecting to Application History server at rbhp-con

### Manual step
1. Update target manager configs in Control (https://dmcontrol-dev.host/admin/target/targetmanagermodel/)
2. Sync ... GO

In [14]:
# Update pipelines for managed projects
for target_obj in ["lookalike", "portrait", "sales_report"]:
    print("Processing {} ...".format(target_obj))

    response = migration.client.get(url="target/{}/".format(target_obj), params={"page_size": 1000000})
    migration.check_response(response, 200)
    objs = response.json()["results"]

    for obj in tqdm.tqdm_notebook(objs, total=len(objs)):
        if obj["status"] in {"pending", "processing"}:
            project = obj.get("project")
            if project:
                migration.update_pipelines(project_name=obj["project"], kill_in_yarn=True)
            else:
                print("object id {} doesn't have attribute 'project'".format(obj["id"]))

    print("Done\n")

Processing lookalike ...



Done

Processing portrait ...


object id 7034 doesn't have attribute 'project'
object id 7033 doesn't have attribute 'project'

Done

Processing sales_report ...



Done



### III. Switch on
Manual actions:
1. For all originally enabled managers - set `project.enabled: true` in manager config.
2. Sync ... GO.
3. Switch on (enable) these managers.
4. Start code below to switch on all previously disabled user projects.

There must be also a flag from https://jira.host/browse/TRG-72710
(https://dmcontrol.host/admin/compose/settingmodel/ `ENGINEERING_WORK: false`)

In [15]:
migration.enable_projects()